In [15]:
seed = 42
proj_id = 'cmat-315920'
root_path  = '/home/jupyter'

%run -i /home/jupyter/MathVGerrmandering_CMAT_2021/redistrict_code.py
pd.set_option('display.max_columns', None)
states = get_states()

opts = {'census_yr':2010,
        'shapes_yr':2020,
        'election_filters' : {
            "race='general' and office='u.s. sen'",
            "race='general' and office='president'",
            "race='general' and office like 'u.s. rep%'"},
        'overwrite' : {
#             'crosswalks',
#             'assignments',
#             'elections',
#             'census',
#             'shapes',
#             'edges',
#             'nodes',
#             'graph',
        }}

# for st in ['TX', 'RI']:
#     for lvl in ['tabblock', 'bg', 'tract']:
#         for dist in ['cd', 'sldu', 'sldl']:
#             opts.update({'abbr':st, 'level':lvl, 'district':dist})
#             self = Gerry(**opts)
#             self.MCMC(steps=0)
#             print('===================================================================================')

opts['overwrite'] = set()
rng = np.random.default_rng(40)
opts.update({'abbr':'TX', 'level':'tabblock', 'district':'cd'})
self = Gerry(**opts)
# self.MCMC(steps=10)
self.steps = 10

self.write_viz_tables()

Creating viz table for step 0
Creating viz table for step 1
Creating viz table for step 2
Creating viz table for step 3
Creating viz table for step 4


KeyboardInterrupt: 